In [11]:
import sys
import os
import dotenv
dotenv.load_dotenv('../../.env')

# Add parent directory to Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','app')))

from app.SIWeaviateClient import SIWeaviateClient


In [3]:
import weaviate.classes as wvc
import weaviate
from weaviate.classes.query import Filter, GeoCoordinate, MetadataQuery, QueryReference, GroupBy


client = weaviate.connect_to_local()
document_chunk = client.collections.get("DocumentChunk")
document = client.collections.get("Document")
doc = document.query.fetch_object_by_id(
    uuid="cbee23ef-8963-4110-9155-232eb66dd870",
    return_references=[QueryReference(
        link_on="hasChunks", 
        return_properties=["title", "meta_page_number"]
    )]
)
chunks = doc.references['hasChunks'].objects
chunks = [{"title": chunk.properties.get('title'), "page": chunk.properties.get('meta_page_number')} for chunk in chunks if chunk.properties.get('title')]

# Remove duplicates, keeping only the first occurrence (min page)
unique_chunks = {}

for chunk in chunks:

    title = chunk['title'].strip().lstrip('>').strip()
    page = chunk['page']
    if title not in unique_chunks or page < unique_chunks[title]['page']:
        unique_chunks[title] = {'title': title, 'page': page}

chunks = list(unique_chunks.values())

for chunk in chunks:
    print(chunk)


client.close()


{'title': 'PREAMBULE', 'page': 4}
{'title': 'Scénario :', 'page': 4}
{'title': 'Dans la régie :', 'page': 5}
{'title': 'Préparation', 'page': 5}
{'title': '2. FICHE', 'page': 6}
{'title': 'DEROULEMENT', 'page': 7}
{'title': 'A faire par les élèves ou le public pendant l’animation :', 'page': 7}
{'title': 'Matériel', 'page': 8}
{'title': 'NIVEAU   ORGANITE', 'page': 23}
{'title': 'Organe :', 'page': 23}
{'title': 'Tissu :', 'page': 23}
{'title': 'Membrane', 'page': 24}
{'title': 'Cytoplasme', 'page': 26}
{'title': 'Organite', 'page': 26}
{'title': 'Mitochondrie', 'page': 26}
{'title': 'Noyau', 'page': 26}
{'title': 'Récepteur', 'page': 26}
{'title': 'Virus', 'page': 27}
{'title': 'Bactérie', 'page': 27}
{'title': 'Molécule', 'page': 28}
{'title': 'Macromolécule', 'page': 28}
{'title': 'ADN', 'page': 30}
{'title': 'RESSOURCES', 'page': 34}
{'title': 'Extraction de l’ADN', 'page': 35}
{'title': '2.  A quoi sert le sel ?', 'page': 36}
{'title': '7. Peux-t-on purifier l’ADN recueilli ?', 'p